# Jupyter Widgets
Simpler GUI for running TARDIS - this demo notebook is a collection of widgets to create complex & powerful interactive interfaces.

In [49]:
from tardis import run_tardis
from tardis.io.atom_data.util import download_atom_data
from tardis.util.base import atomic_number2element_symbol, species_tuple_to_string

import pandas as pd
import numpy as np
import qgrid
import ipywidgets as ipw

## Create Model

In [3]:
# Using configuration file stored in same directory as of notebook
sim = run_tardis('tardis_example.yml')

[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:74)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/jals/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:29)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:184)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data (base.py:187)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:268)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.93730e+42 erg / s Luminosity absorbed = 2.66400e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:359)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    next_t_rad         w    next_w
	Shell                                               
	0 

[tardis.simulation.base][INFO   ]  t_inner 10025.810 K -- next t_inner 11278.874 K (base.py:352)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:268)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.32624e+43 erg / s Luminosity absorbed = 4.34986e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:359)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10590.775937  11489.769556  0.439574  0.511403
	5      11060.842307  11791.517605  0.154917  0.191127
	10     10761.665022  11527.732208  0.099840  0.120106
	15     10436.629671  11111.114390  0.074676  0.091713

 (base.py:350)
[tardis.simulation.base][INFO   ]  t_inner 11278.874 K -- next t_inner 10079.967 K (base.py:352)
[tardis.simulation.base][INFO   ]  Starting iteration 11/20 (base.py:268)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.51915e+42 erg / s Lum

[tardis.simulation.base][INFO   ]  t_inner 10245.337 K -- next t_inner 11056.949 K (base.py:352)
[tardis.simulation.base][INFO   ]  Starting iteration 20/20 (base.py:268)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.22623e+43 erg / s Luminosity absorbed = 4.00603e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:359)
[tardis.simulation.base][INFO   ]  Simulation finished in 20 iterations and took 40.23 s (base.py:308)


## Extract Data of all 4 tables 
Shell temp., Z count, Ion count, Level count)

In [4]:
shells_data = (pd.DataFrame({'Rad. Temp': sim.model.t_rad,
                            'Ws': sim.model.w}, index=range(1, 21))
               .applymap(lambda x: '{:.6e}'.format(x))) # to make qgrid show values in scientific notation
shells_data.index.name = 'Shell No.'
shells_data

,Rad. Temp,Ws
Shell No.,,
1,1.083473e+04,4.383219e-01
2,1.102452e+04,3.267770e-01
3,1.113486e+04,2.612892e-01
4,1.129932e+04,2.077030e-01
5,1.130407e+04,1.772559e-01
6,1.134613e+04,1.542037e-01
7,1.127387e+04,1.387194e-01
8,1.120133e+04,1.264399e-01
9,1.108035e+04,1.183038e-01


In [70]:
def Z_count(shell_num):
    Z_count_data = sim.plasma.abundance[shell_num-1]
    return pd.DataFrame({
        'Element': Z_count_data.index.map(atomic_number2element_symbol),
        # To make qgrid show values in scientific notation convert them to string
        'Count (Shell {})'.format(shell_num): Z_count_data.map('{:.6e}'.format)
    })

Z_count(11)

,Element,Count (Shell 11)
Z,,
8,O,1.900000e-01
12,Mg,3.000000e-02
14,Si,5.200000e-01
16,S,1.900000e-01
18,Ar,4.000000e-02
20,Ca,3.000000e-02


In [69]:
def ion_count(Z, shell_num):
    ion_num_density = sim.plasma.ion_number_density[shell_num-1].loc[Z]
    Z_num_density = sim.plasma.number_density.loc[Z, shell_num-1]
    ion_count_data = ion_num_density/Z_num_density  # Normalization
    ion_count_data.index.name = 'Ion'
    return pd.DataFrame({
        'Species': ion_count_data.index.map(lambda x: species_tuple_to_string((Z, x))),
        'Count (Z={})'.format(Z): ion_count_data.map('{:.6e}'.format, na_action='ignore')
    })


ion_count(8, 2)

,Species,Count (Z=8)
Ion,,
0,O I,1.475612e-06
1,O II,9.995335e-01
2,O III,4.650015e-04
3,O IV,5.719922e-17
4,O V,0.000000e+00
5,O VI,0.000000e+00
6,O VII,0.000000e+00
7,O VIII,0.000000e+00
8,O IX,0.000000e+00


In [71]:
def level_count(ion, Z, shell_num):
    level_num_density = sim.plasma.level_number_density[shell_num-1].loc[Z, ion]
    ion_num_density = sim.plasma.ion_number_density[shell_num-1].loc[Z, ion]
    level_count_data = level_num_density/ion_num_density  # Normalization
    level_count_data.index.name = 'Level'
    level_count_data.name = 'Count (Ion {})'.format(ion)
    return level_count_data.map('{:.6e}'.format, na_action='ignore').to_frame()


level_count(3, 12, 2)

,Count (Ion 3)
Level,
0,2.668312e-01
1,5.336624e-01
2,1.995065e-01
3,5.875975e-19
4,1.219322e-31
5,6.756197e-32
6,3.036132e-32
7,2.219831e-32
8,8.918837e-33


## Create Table Widgets

### Create qgrid Table widgets and set right options

In [285]:
grid_options = {'sortable': False,
                'filterable': False,
                'editable': False,
                'minVisibleRows': 2,
                }

column_options = {
    'minWidth': None,
}


def column_widths_definitions(df, col_widths):
    '''
    Generate column definition dictionary from the widths specified for 
    each column (col_widths) including index as a column, in a dataframe (df)
    '''
    cols_with_index = [df.index.name] + df.columns.to_list()
    return {col_name: {'width': col_width}
            for col_name, col_width in zip(cols_with_index, col_widths)}


# Since forceFitColumns is enabled by default, the column widths (when all specfied)
# get applied in proportions, despite their original unit is px
shells_col_widths = [30, 35, 35]  # these values are proportions of 100
shells_table = qgrid.show_grid(shells_data,
                               grid_options=grid_options,
                               column_options=column_options,
                               column_definitions=column_widths_definitions(
                                   shells_data,
                                   shells_col_widths
                               ))

Z_count_shell1 = Z_count(1)
Z_count_col_widths = [20, 30, 50]
Z_column_widths_definitions = column_widths_definitions(
    Z_count_shell1,
    Z_count_col_widths
)
for shell_num in range(1, 21):
    Z_column_widths_definitions['Count (Shell {})'.format(
        shell_num)] = {'width': Z_count_col_widths[-1]}
Z_count_table = qgrid.show_grid(Z_count_shell1,
                                grid_options=grid_options,
                                column_options=column_options,
                                column_definitions=Z_column_widths_definitions)


ion_count_Z8_shell1 = ion_count(8, 1)
ion_count_col_widths = [20, 30, 50]
ion_column_widths_definitions = column_widths_definitions(
    ion_count_Z8_shell1,
    ion_count_col_widths
)
for Z in Z_count_shell1.index:
    ion_column_widths_definitions['Count (Z={})'.format(
        Z)] = {'width': ion_count_col_widths[-1]}
ion_count_table = qgrid.show_grid(ion_count_Z8_shell1,
                                  grid_options=grid_options,
                                  column_options=column_options,
                                  column_definitions=ion_column_widths_definitions)


level_count_ion0_Z8_shell1 = level_count(0, 8, 1)
level_count_col_widths = [40, 60]
level_column_widths_definitions = column_widths_definitions(
    level_count_ion0_Z8_shell1,
    level_count_col_widths
)
for level in range(0, 21):
    level_column_widths_definitions['Count (Ion {})'.format(
        level)] = {'width': level_count_col_widths[-1]}
level_count_table = qgrid.show_grid(level_count_ion0_Z8_shell1,
                                    grid_options=grid_options,
                                    column_options=column_options,
                                    column_definitions=level_column_widths_definitions)

out = ipw.Output()

### Event Listeners
(JS part)

In [94]:
# Using trailet observe

# def update_Z_count_table(change):
#     shell_num = change.new[0]+1
#     Z_count_table.df = Z_count(shell_num)
#     with out:
#         print('Shell selected is {}'.format(shell_num))

# shells_table.observe(update_Z_count_table, names=['_selected_rows'])

In [286]:
# Using qgridWidget.on

def update_Z_count_table(event, qgrid_widget):
    # Get shell number from row selected in shells_table
    shell_num = event['new'][0]+1
    
    # Update data in Z_count_table
    Z_count_table.df = Z_count(shell_num)
#     with out:
#         print('Due to SHELL TABLE: Shell {}'.format(shell_num))
#         print('Old rows selected in Z table {}'.format(
#             Z_count_table.get_selected_rows()))
    
    # Since name of last column gets changed when updating data,
#     # we need to specify column width for this new column name
#     if Z_count_table.df.columns[-1] not in Z_count_table.column_definitions:
#         Z_count_table.column_definitions[Z_count_table.df.columns[-1]] = {
#             'width': Z_count_col_widths[-1]}
    
    # Get Z of 0th row of Z_count_table
    Z0 = Z_count_table.df.index[0]
#     Z = Z_count(shell_num).index[0]

    # [0] -> [] -> [0] to trigger
    # update_ion_count_table won't trigger if last row selected in Z_count_table was also 0th
    if Z_count_table.get_selected_rows() == [0]:
#         with out:
#             print('Within if: rows selected {}'.format(
#                 Z_count_table.get_selected_rows()))

        # Unselect rows
        Z_count_table.change_selection([])

    # Select 0th row in count table which will also trigger update_ion_count_table
    Z_count_table.change_selection([Z0])

    # Alternatively: by using a dummy event dict (Not possible now due to a bug in Qgrid)
#     if Z_count_table.get_selected_rows()==[0]: #Ion count table update won't trigger if last selected Z table row was also 0th
#         event = {'name': 'selection_changed', 'old': [], 'new': [0], 'source': 'api'}
#         on_Zcount_row_selected(event, qgrid_widget)
#     else:
#         Z_count_table.change_selection([Z])

#     with out:
#         print('New rows selected in Z table {}\n'.format(
#             Z_count_table.get_selected_rows()))


shells_table.on('selection_changed', update_Z_count_table)

In [287]:
def update_ion_count_table(event, qgrid_widget):
#     with out:
#         print(event)

    # Don't execute function if no row was selected implicitly (by api)
    if event['new'] == [] and event['source'] == 'api':
        return

    shell_num = shells_table.get_selected_rows()[0]+1
    Z = Z_count_table.df.index[event['new'][0]]

    ion_count_table.df = ion_count(Z, shell_num)
#     with out:
#         print('Due to Z_COUNT TABLE: Shell {}, Z {}'.format(shell_num, Z))
        
    ion0 = ion_count_table.df.index[0]
    if ion_count_table.get_selected_rows()==[0]:
        ion_count_table.change_selection([])
    ion_count_table.change_selection([ion0])


Z_count_table.on('selection_changed', update_ion_count_table)

In [288]:
def update_level_count_table(event, qgrid_widget):
    
    # Don't execute function if no row was selected implicitly (by api)
    if event['new'] == [] and event['source'] == 'api':
        return
    
    shell_num = shells_table.get_selected_rows()[0]+1
    Z = Z_count_table.df.index[Z_count_table.get_selected_rows()[0]]
    ion = ion_count_table.df.index[event['new'][0]]

    level_count_table.df = level_count(ion, Z, shell_num)
    


ion_count_table.on('selection_changed', update_level_count_table)

### Styling the main layout 
CSS part - To make all components look nicely together

In [289]:
box_layout = ipw.Layout(display='flex',
                        # flex_flow='row',
                        align_items='flex-start',
                        justify_content='space-between',
                        # width='100%',
                        # border='1px solid yellow'
                        )

item_layout = ipw.Layout(width='24%',
                         # border='1px solid red'
                         )

shells_table.layout.width = '32%'
Z_count_table.layout.width = '24%'
ion_count_table.layout.width = '24%'
level_count_table.layout.width = '18%'

# out.layout.height='500px'
out.layout.border = '1px solid black'

# Z_count_table.layout.align_self = 'center'

In [93]:
# Style specific Qgrid properties
# level_count_table.change_grid_option('minVisibleRows', '2')

### Display

In [290]:
modelparam = '''Iterations requested: {}\nIterations executed:  {}
Model converged     : {}\nSimulation Time    :  {} s
Inner Temperature   : {} K\nNumber of packets  :  {}
Inner Luminosity    : {}'''.format(sim.iterations,
                        sim.iterations_executed,
                        'True' if sim.converged else 'False',
                        sim.runner.time_of_simulation.value,
                        sim.model.t_inner.value,
                        sim.last_no_of_packets,
                        sim.runner.calculate_luminosity_inner(sim.model))
# modelparam

In [291]:
shell_info_tables = ipw.Box([shells_table, Z_count_table, ion_count_table, level_count_table],
                  layout=box_layout)
shells_table.change_selection([1])

model_parameters = ipw.Output(layout={'border': '1px solid black', 'width': '920px'})
with model_parameters:
    print(modelparam)
    
shell_info = ipw.Tab(children=[shell_info_tables, model_parameters])
shell_info.set_title(0, 'Shells')
shell_info.set_title(1, 'Model Parameters')
display(shell_info)